In [2]:
import pandas as pd
import numpy as np
import numpy as np
import torch
import os
import multiprocessing
from multiprocessing import Pool
from collections import Counter

In [ ]:
#loading all the human sequence
#objective: from seq fle to fasta file
human_seq_df = pd.read_table("/home/sxr280/DeepRBPLoc/new_data/All RNA sequence/human_RNA_sequence.txt", index_col = None).iloc[:100,:]
# for line in range(human_seq_df.shape[0]):
#     geneid = human_seq_df.iloc[line,:]["Gene_ID"]


# info_df = pd.read_table("/home/sxr280/DeepRBPLoc/new_data/All_RNA_subcellular_localization_data.txt")
class preprocess:
    def __init__(self, seq_path, info_path):
        self.seq_path = seq_path
        self.info_path = info_path
        info_df = pd.read_table(info_path)
        self.info_df = info_df
        # self.fasta_path = fasta_path

    def get_tag(self, labels : list):
        for idx,label in enumerate(labels):
            if label in ["Nucleus", "Chromatin" , "Nucleoplasm", "Nucleolus", "Nuclear"]:
                labels[idx] = "Nucleus"
        refs = np.array(["Nucleus","Exosome","Cytosol","Cytoplasm","Ribosome","Membrane","Endoplasmic reticulum", "Microvesicle", "Mitochondrion"])
        # y = np.array(labels == label, dtype = "int")  
        y = ["1" if i in labels else "0" for i in refs]
        return y
    def process_row(self,row, fasta_str_lst):
        # print(type(row))
        row = row[1]
        # fasta_str = ""
        geneid = row["Gene_ID"] 
        seq = row["Sequence"] 
        refid = row["Refseq_ID"]
        species = self.info_df[self.info_df["Gene_ID"] == geneid]["Species"].values[0]
        RNA_cate = self.info_df[self.info_df["Gene_ID"] == geneid]["RNA_category"].values[0]
        locs = list(self.info_df[self.info_df["Gene_ID"] == geneid]["SubCellular_Localization"].values)
        tag = "".join(self.get_tag(locs))
        fasta_str = ">" + tag + ",Gene_ID:" + geneid + ",Refseq_ID:" + refid + ",Species:" + species + ",RNA_category:" + RNA_cate + "\n" + seq + "\n"
        fasta_str_lst.append(fasta_str)
        return fasta_str_lst

        seq_df = seq_df.apply(modify_seq_df, axis=1)
    def df2fasta(self, fasta_path, pool_num):
        seq_df = pd.read_table(self.seq_path, index_col = None)
        info_df = pd.read_table("/home/sxr280/DeepRBPLoc/new_data/All_RNA_subcellular_localization_data.txt")
        tags = []

        #using multiprocessing
        manager = multiprocessing.Manager()
        fasta_str_lst = manager.list()
        jobs = []
        p = Pool(pool_num)

        for row in seq_df.iterrows():
            job=p.apply_async(self.process_row,args=(row, fasta_str_lst))
            jobs.append(job)
        p.close()
        p.join()
        print("after getting all the fasta string")

        fasta_str = "".join(fasta_str_lst)
        fasta_dir, filename = os.path.split(fasta_path)
        if not os.path.exists(fasta_dir):
            print("auto creating the fasta file path")
            os.makedirs(fasta_dir)
        with open(fasta_path, "w") as f1:
            print("input the data to fasta file")
            f1.write(fasta_str)


info_path = "/home/sxr280/DeepRBPLoc/new_data/All_RNA_subcellular_localization_data.txt"
seq_path = "/home/sxr280/DeepRBPLoc/new_data/All RNA sequence/human_RNA_sequence.txt"
fasta_path = "/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq.fasta"
pro = preprocess(seq_path, info_path, pool_num = 10)
pro.df2fasta(fasta_path)




# Statistics of the data

In [ ]:
#what kinds of RNA we have in the test dataset

In [1]:
import re
import numpy as np
import pandas as pd
seq_lst = []

# with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq.fasta", "r") as f1:
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2_filtermilnc.fasta", "r") as f1:
    for line in f1.readlines():
        # print(line)
        if line.startswith(">") == False:
            # print(line)
            seq_lst.append(len(line.strip()))
        # else:
        #     RNA_type = line.split(":")[-1].strip()
    # print(string[:10])
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2_filtermilnc.fasta", "r") as f1:

    string = f1.read()
pattern = r">(\d*).*Gene_ID:\w+:(\w*\d+)?,.*RNA_category:([^\n,]+)"
# pattern_seq = r"[ATGCNU][ATGCNU].+\n"
# print(len(seq_lst))
matches = np.array(re.findall(pattern, string))
# matches_seq = np.array(re.findall(pattern_seq, string))
# print(matches[:3])
refs = np.array(["Nucleus","Exosome","Cytosol","Cytoplasm","Ribosome","Membrane","Endoplasmic reticulum", "Microvesicle", "Mitochondrion"])
loc_rna = {}
rna_loc = {}
id_seq = {}
id_type = {}
for idx,i in enumerate(matches):
    # print(idx)
    seq_len = seq_lst[idx]
    tag = i[0]
    ncbi_id = i[1]
    # print(ncbi_id)
    RNA_type = i[2]
    # print(tag)
    id_seq.setdefault(ncbi_id, []).append(seq_len)
    id_type.setdefault(ncbi_id, []).append(RNA_type)
    locs_idx = [idx for idx, j in enumerate(tag) if j == "1"]
    # print(locs_idx)
    for loc in refs[locs_idx]:
        loc_rna.setdefault(loc,[]).append(RNA_type)
        rna_loc.setdefault(RNA_type,[]).append(loc+"_"+ncbi_id)
# print(loc_rna["Exosome"])

comps = []
rnas = []
for key, value in loc_rna.items():
    # print(list(key) * len(value))
    comps += [key] * len(value)
    rnas += list(value)

# Convert the new dictionary to a Pandas dataframe
df = pd.DataFrame({"Compartments": comps, "RNA_type": rnas, "Values": [1]*len(rnas)})
df_seq = pd.DataFrame({"ID": list(id_seq.keys()), "Length": [max(i) for i in id_seq.values()], "RNA_type": [i[0] for i in id_type.values()]})

# Print the dataframe
# print(df)

#count the compartement of each RNA type
for k,v in rna_loc.items():
    print("Calculating:", k)
    v = set(v)
    v1_list = [v1.split("_")[0] for v1 in v]
    print(Counter(v1_list))

# print(df_seq)


IsADirectoryError: [Errno 21] Is a directory: '/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2_filtermilnc'

In [ ]:
a = [1,2,3]
max(a) 

In [ ]:
%%capture
%load_ext rpy2.ipython

In [ ]:
%R -i df -i df_seq

In [ ]:
%%R -w 1200 -h 500
library(ggplot2)
library(dplyr)
# miRNA_data$Classes <- as.factor(miRNA_data$Classes)
data_summary <- df %>%
  group_by(Compartments, RNA_type) %>%
  summarise(count = n()) %>%
  group_by(Compartments) %>%
  mutate(percentage = count / sum(count) * 100)
print(data_summary)
Theme <- theme(panel.grid = element_blank(),  # Remove gridlines
        panel.background = element_blank(),
        axis.line = element_line(color = "black"),
        axis.text = element_text(family = "Arial", size = 12))
# title = sprintf("attention weight visualization - mRNA - support: %s", all_support_num)
custom_colors <- c("#00468BFF", "#FDAF91FF", "#EE4C97FF", "#ED0000FF", "#42B540FF", "#0099B4FF", "925E9FFF", "#4DBBD5FF", "#00A087FF", "#8491B4FF", "#91D1C2FF", "#7E6148FF")
ggplot(data_summary, aes(x = Compartments, y = percentage, fill = factor(RNA_type))) +
  geom_bar(stat = "identity", position = "stack") +
  labs(title = "test",
       x = "Compartments",
       y = "Percentages") + 
       scale_fill_manual(values = custom_colors) + Theme

# ggsave("/home/sxr280/DeepRBPLoc/new_data/fig/miRNA_fine_tune_lineplot_roauc.png", type = "cairo-png", dpi = 300)



In [ ]:
#Statistic the sequence DM3Loc they use
with open("/home/sxr280/DeepRBPLoc/testdata/modified_multilabel_seq.fasta", "r") as f1:
    symbol = ""
    for line in f1.readlines():
        if line[0] == ">":
            # print(symbol)
            if symbol == "COL13A1":
                print(len(seq))
            symbol = line.split("Gene_Name:")[1].strip()
            
            
        else:
            seq = line



In [ ]:
with open("/home/sxr280/DeepRBPLoc/new_data/GRCh38_latest_rna.fna", "r") as f1:
    symbol = ""
    len_list = []
    for line in f1.readlines():
        if line[0] == ">":
            # print(symbol)
            if symbol == "COL13A1":
                # print(len(seq))
                len_list.append(len(seq))
            symbol = line.split("(")[1].split(")")[0]
            seq = ""
            
            
        else:
            seq += line.strip()
max(len_list)

# Deduplication of the sequence file (the longest are selected)

In [ ]:
import os
def deduplicated(file_path):
    with open(file_path, "r") as f1:
    # with open("/home/sxr280/DeepRBPLoc/new_data/GRCh38_latest_rna.fna", "r") as f1:
        ncbiid = ""
        id_lst = []
        len_list = []
        new_str = ""
        id_seq = {}
        id_header = {}
        all_num = 0
        filtered_num = 0
        for line in f1.readlines():
            if line[0] == ">":
                # if id_lst.count(ncbiid) == 1:
                #     # if ncbiid not in id_lst:
                #     new_str += header+seq
                #     filtered_num += 1
                # else:
                if len(ncbiid) > 0:
                    id_seq.setdefault(ncbiid, []).append(seq)
                    id_header.setdefault(ncbiid, []).append(header)

                ncbiid = line.split("Gene_ID:")[1].split(",")[0].replace("NCBI","")
                id_lst.append(ncbiid)
                header = line
                seq = ""
                all_num += 1
                
                
            else:
                seq += line

    new_str = ""
    import numpy as np
    for ncbiid in list(id_seq.keys()):
        seq_lst = id_seq[ncbiid]
        header_lst = id_header[ncbiid]
        max_seq = max(seq_lst, key = lambda x: len(x))
        # print(len(max(seq_lst, key = lambda x: len(x))))
        max_idx = np.where(np.array(seq_lst) == max_seq)[0][0]
        print(max_idx)
        if max_idx != 0:
            print(max_idx)
        # max_seq = max(seq_lst, key = lambda x: len(x))
        max_header = header_lst[max_idx]
        new_str += max_header+max_seq
    base_dir = os.path.dirname(file_path)
    file_name = os.path.basename(file_path).split(".")[0]
    with open(base_dir + "/" + file_name + "_deduplicated.fasta", "w") as f2:
        f2.write(new_str)
# deduplicated("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq.fasta")

deduplicated("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2.txt")
# deduplicated("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_mouse_data_seq.fasta")
# deduplicated("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_otherspecies_data_seq.fasta")

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
3
3
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
2
2
0
0
0
0
8
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
6
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
dm3loc_c = 0
mRNA_c = 0
new_str = ""
tag = 0
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated.fasta", "r") as f2:

    for line in f2.readlines():
        if line[0] == ">":
            source = line.split(":")[-1].strip()
            rna = line.split("RNA_category:")[1].split(",")[0]
            
            if rna == "mRNA":
                mRNA_c +=1
                if source == "DM3Loc":
                    dm3loc_c += 1
                    new_str += line
                    tag = 1
                else:
                    tag = 0
            else:
                new_str += line
                tag = 1
        else:
            # seq = line.strip()
            if tag != 0:
                new_str += line
                tag = 0

with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2.fasta", "w") as f3:
    f3.write(new_str)

print(dm3loc_c)
print(mRNA_c)

#remove the allseq mRNA


17304
17938


In [ ]:
dm3loc_c = 0
mRNA_c = 0
new_str = ""
tag = 0
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2.fasta", "r") as f2:

    for line in f2.readlines():
        if line[0] == ">":
            source = line.split(":")[-1].strip()
            rna = line.split("RNA_category:")[1].split(",")[0]
            
            if rna == "mRNA":
                mRNA_c +=1
                if source == "DM3Loc":
                    dm3loc_c += 1
                    new_str += line
                    tag = 1
                else:
                    tag = 0
            else:
                new_str += line
                tag = 1
        else:
            # seq = line.strip()
            if tag != 0:
                new_str += line
                tag = 0
print(dm3loc_c)
print(mRNA_c)

17304
17304


# Merge the file with DM3Loc (I want to repeat the sequence that come from DM3Loc for easily comparison)

In [ ]:
#loading the deuplicatated file from dm3loc
symbol = ""
id_seq_dm3loc = {}
id_header_dm3loc = {}
with open("/home/sxr280/DeepRBPLoc/testdata/modified_multilabel_seq_nonredundent.fasta") as f1:
    for line in f1.readlines():
        if line[0] == ">":
            if len(symbol) != 0:
                id_seq_dm3loc.setdefault(geneid, []).append(seq)
                id_header_dm3loc.setdefault(geneid, []).append(header)
            header = line.strip()
            tag = line.split(",")[0]
            refid = line.split(",")[1]
            geneid = line.split(",")[2].split("Gene_ID:")[1]
            symbol = line.split(",")[3].split("Gene_Name:")[1].strip()
            
        else:
            seq = line.strip()


In [ ]:
len(id_header_dm3loc.keys())

17305

In [ ]:
#loading the extracted redandant file and do the merge
geneid = ""
id_seq_allseq = {}
id_header_allseq = {}
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq.fasta", "r") as f1:
    for line in f1.readlines():
        if line[0] == ">":
            if len(geneid) != 0:
                id_seq_allseq.setdefault(geneid, []).append(seq)
                id_header_allseq.setdefault(geneid, []).append(header)
            header = line.strip() + ",Source:AllSeq"
            tag = line.split(",")[0]
            refid = line.split(",")[2]
            geneid = line.split(",")[1].replace("Gene_ID:NCBI:", "").replace("Gene_ID:miRBase:", "")
            # print(geneid)
            # symbol = line.split(",")[3].split("Gene_Name:")[1].strip()
        else:
            seq = line.strip()

In [ ]:
# refs = np.array(["Nucleus","Exosome","Cytosol","Cytoplasm","Ribosome","Membrane","Endoplasmic reticulum", "Microvesicle", "Mitochondrion"])


In [ ]:
#iterating the 

#interate all seq dataset
# for geneid,seq_lst in id_seq_allseq.items():
#     header_allseq = id_header_allseq[geneid]
#     tag_allseq = header_allseq.split(",")[0][1:]
#     geneid_allseq = header_allseq.split(",")[1]
#     refid_allseq = header_allseq.split(",")[2]
#     species_allseq = header_allseq.split(",")[3]
#     rnacaate_allseq = header_allseq.split(",")[4].strip()
    # if geneid in id_seq_dm3loc.keys():

#iterate dm3loc dataset
    
out_c = 0
in_c = 0
seq_in = 0
seq_out = 0
for geneid,seq_lst_dm3loc in id_seq_dm3loc.items():
    header_dm3loc_lst = id_header_dm3loc[geneid]
    new_header_lst = []
    for header_dm3loc in header_dm3loc_lst:
        tag_align = header_dm3loc.split(",")[0] + "00"
        refid_align = "Refseq_ID:" + header_dm3loc.split(",")[1].split(":")[1]
        geneid_align = header_dm3loc.split(",")[2].split(":")[0]+":NCBI:"+header_dm3loc.split(",")[2].split(":")[1]
        symbol_align = header_dm3loc.split(",")[3].strip()
        new_header = tag_align + "," + geneid_align + "," + refid_align + ",Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc"
        new_header_lst.append(new_header)
    if geneid in list(id_seq_allseq.keys()):
        in_c += 1
        seq_lst_allseq =  id_seq_allseq[geneid]
        for idx,seq in enumerate(seq_lst_dm3loc):
            new_header = new_header_lst[idx]
            if seq not in seq_lst_allseq:
                seq_out += 1

                # print(new_header, "adding the new seq from dm3loc to allseq")
                id_header_allseq[geneid] = = new_header_lst
                #dm3loc substitute allseq
                # id_header_allseq[geneid] = new_header_lst
                # id_seq_allseq[geneid] = seq_lst_dm3loc
                print(new_header)
                id_seq_allseq[geneid].append(seq)
            else:
                seq_in += 1
                # print(new_header,"already in the all seq")
    else:
        id_header_allseq[geneid] = new_header_lst
        id_seq_allseq[geneid] = seq_lst_dm3loc
        out_c += 1
print(out_c,"newly added symbol from dm3loc")
print(in_c,"already exist in all seq")
print(seq_in, "sequence already in the all seq")
print(seq_out, "sequence not in the all seq")






   
        

>110000000,Gene_ID:NCBI:3761,Refseq_ID:NM_004981,Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc
>010000000,Gene_ID:NCBI:3885,Refseq_ID:NM_021013,Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc
>110000000,Gene_ID:NCBI:434,Refseq_ID:NM_001672,Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc
>110100000,Gene_ID:NCBI:51263,Refseq_ID:NM_016503,Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc
>010000000,Gene_ID:NCBI:3743,Refseq_ID:NM_031886,Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc
>111100000,Gene_ID:NCBI:388591,Refseq_ID:NM_173795,Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc
>010000000,Gene_ID:NCBI:342372,Refseq_ID:NM_181536,Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc
>110000000,Gene_ID:NCBI:5021,Refseq_ID:NM_000916,Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc
>111110000,Gene_ID:NCBI:29956,Refseq_ID:NM_013384,Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc
>110001000,Gene_ID:NCBI:347918,Refseq_ID:NM_182613,Species:Homo sapiens,RNA_c

In [ ]:
#new version

#interate all seq dataset
# for geneid,seq_lst in id_seq_allseq.items():
#     header_allseq = id_header_allseq[geneid]
#     tag_allseq = header_allseq.split(",")[0][1:]
#     geneid_allseq = header_allseq.split(",")[1]
#     refid_allseq = header_allseq.split(",")[2]
#     species_allseq = header_allseq.split(",")[3]
#     rnacaate_allseq = header_allseq.split(",")[4].strip()
    # if geneid in id_seq_dm3loc.keys():

#iterate dm3loc dataset
    
out_c = 0
in_c = 0
seq_in = 0
seq_out = 0
for geneid,seq_lst_dm3loc in id_seq_dm3loc.items():
    header_dm3loc_lst = id_header_dm3loc[geneid]
    new_header_lst = []
    for header_dm3loc in header_dm3loc_lst:
        tag_align = header_dm3loc.split(",")[0] + "00"
        refid_align = "Refseq_ID:" + header_dm3loc.split(",")[1].split(":")[1]
        geneid_align = header_dm3loc.split(",")[2].split(":")[0]+":NCBI:"+header_dm3loc.split(",")[2].split(":")[1]
        symbol_align = header_dm3loc.split(",")[3].strip()
        new_header = tag_align + "," + geneid_align + "," + refid_align + ",Species:Homo sapiens,RNA_category:mRNA,Source:DM3Loc"
        new_header_lst.append(new_header)


    id_header_allseq[geneid] = new_header_lst
    id_seq_allseq[geneid] = seq_lst_dm3loc
    out_c += 1
print(out_c,"newly added symbol from dm3loc")
# print(in_c,"already exist in all seq")
# print(seq_in, "sequence already in the all seq")
# print(seq_out, "sequence not in the all seq")






   
        

17305 newly added symbol from dm3loc


In [ ]:
#new version
merge_str = ""
for geneid,seq_lst in id_seq_allseq.items():
    header_lst = id_header_allseq[geneid]
    tag_lst = []
    for header, seq in zip(header_lst, seq_lst):
        
        tag = header.split(",")[0]
        tag_lst.append(tag)
        tag_set = list(set(tag_lst))
        if ">" not in header:
            header = ">" + header
        
        merge_str += header+"\n"+seq+"\n"
        
       
        
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2.txt","w") as f4:
    f4.write(merge_str)

In [ ]:
merge_str = ""
for geneid,seq_lst in id_seq_allseq.items():
    header_lst = id_header_allseq[geneid]
    tag_lst = []
    for header, seq in zip(header_lst, seq_lst):
        
        tag = header.split(",")[0]
        tag_lst.append(tag)
        tag_set = list(set(tag_lst))
        if ">" not in header:
            header = ">" + header
        if len(tag_set) > 1:

            header = header.replace(tag,  tag_lst[0])
            if "11016" in geneid:
                # print(tag_lst)
                print("from",header.split(",")[0],"to", tag_lst[0], tag_set)
                print(header)
            merge_str += header+"\n"+seq+"\n"
        else:
            merge_str += header+"\n"+seq+"\n"
        if "11016" in geneid:
                # print(tag_lst)
                # print("from",header.split(",")[0],"to", tag_lst[0], tag_set)
            print(header)
        
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2.txt","w") as f4:
    f4.write(merge_str)

>111000000,Gene_ID:NCBI:11016,Refseq_ID:NR_073163.2,Species:Homo sapiens,RNA_category:mRNA,Source:AllSeq
>111000000,Gene_ID:NCBI:11016,Refseq_ID:NM_001206683.1,Species:Homo sapiens,RNA_category:mRNA,Source:AllSeq
>111000000,Gene_ID:NCBI:11016,Refseq_ID:NM_001366561.2,Species:Homo sapiens,RNA_category:mRNA,Source:AllSeq
>111000000,Gene_ID:NCBI:11016,Refseq_ID:NM_001130060.2,Species:Homo sapiens,RNA_category:mRNA,Source:AllSeq
>111000000,Gene_ID:NCBI:11016,Refseq_ID:NM_001366555.2,Species:Homo sapiens,RNA_category:mRNA,Source:AllSeq
>111000000,Gene_ID:NCBI:11016,Refseq_ID:NM_001206682.2,Species:Homo sapiens,RNA_category:mRNA,Source:AllSeq
>111000000,Gene_ID:NCBI:11016,Refseq_ID:NM_001366563.2,Species:Homo sapiens,RNA_category:mRNA,Source:AllSeq
>111000000,Gene_ID:NCBI:11016,Refseq_ID:NM_001366562.2,Species:Homo sapiens,RNA_category:mRNA,Source:AllSeq
>111000000,Gene_ID:NCBI:11016,Refseq_ID:NM_006856.3,Species:Homo sapiens,RNA_category:mRNA,Source:AllSeq
>111000000,Gene_ID:NCBI:11016,Refs

# Filtering the dataset with only mRNA to do the benchmarking

In [ ]:
import numpy as np
for fold in range(5):  
    for s in ["Train", "Test", "Val"]:
        X_tag = np.load("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2/%s_fold%s_X_tag.npy" % (s, fold))
        mRNA_site = np.where(X_tag[:,0] == 9)[0]#get the mRNA sites
        X = np.load("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2/%s_fold%s_X.npy" % (s, fold))
        y = np.load("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2/%s_fold%s_y.npy" % (s, fold))
        mask = np.load("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2/%s_fold%s_mask.npy" % (s, fold))
        #get the mRNA dataset
        X_mRNA = X[mRNA_site]
        y_mRNA = y[mRNA_site]
        mask_mRNA = mask[mRNA_site]
        #save the selected mRNA dataset
        np.save("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2/%s_fold%s_X_mRNA.npy" % (s, fold), X_mRNA)
        np.save("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2/%s_fold%s_y_mRNA.npy" % (s, fold), y_mRNA)
        np.save("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2/%s_fold%s_mask_mRNA.npy" % (s, fold), mask_mRNA)
    

In [ ]:
X = np.load("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall_deduplicated/%s_fold%s_X.npy" % ("Train", 0))
X_tag = np.load("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall_deduplicated/%s_fold%s_X_tag.npy" % ("Train", 0))

print(X)
print(X_tag)

[[1 4 3 ... 0 0 0]
 [1 3 1 ... 0 0 0]
 [4 2 2 ... 0 0 0]
 ...
 [3 4 3 ... 0 0 0]
 [3 2 2 ... 0 0 0]
 [3 4 1 ... 0 0 0]]
[[ 9  1  4 ...  0  0  0]
 [ 9  1  3 ...  0  0  0]
 [ 9  4  2 ...  0  0  0]
 ...
 [16  3  4 ...  0  0  0]
 [ 9  3  2 ...  0  0  0]
 [ 8  3  4 ...  0  0  0]]


In [ ]:
X_tag = np.load("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall_deduplicated/%s_fold%s_X_tag.npy" % ("Train", 2))
mRNA_site = np.where(X_tag[:,0] == 9)[0]#get the mRNA sites
X = np.load("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall_deduplicated/%s_fold%s_X_mRNA.npy" % ("Train", 2))
y = np.load("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall_deduplicated/%s_fold%s_y_mRNA.npy" % ("Train", 2))


In [ ]:
# check the test set
X = np.load("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall_deduplicated/Test_fold0_X.npy")


# getting the mRNA sequences


In [ ]:
new_str = ""
tag = 0
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall_deduplicated.fasta", "r") as f1:
    for line in f1.readlines():
        if line[0] == ">":
            types = line.split(",")[-2].split(":")[1]
            if types == "mRNA":
                new_str += line
                print(types)
                tag = 1
            else:
                tag = 0
        else:
            if tag == 1:
                new_str += line
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/mRNA_all_human_data_seq_mergedm3locall_deduplicated.fasta", "w") as f2:
    f2.write(new_str)

mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA
mRNA


In [ ]:
#

# check the label distribution

In [ ]:
import pickle
train = pickle.load(open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2/Train50.pkl","rb"))
train_tag = [i.split("RNA_category:")[1].split(",")[0] for i in train if i[1:].split(",")[0] == "010000000"]
print(set(train_tag))

{'pseudo', 'mRNA', 'ncRNA', 'snoRNA', 'snRNA', 'lncRNA', 'scRNA', 'lincRNA', 'miRNA', 'rRNA'}


In [14]:
import pickle
import re
def GetRNAtype(dataset):
    with open(dataset, "r") as f1:
        string = f1.read()
    
    pattern = r"RNA_category:([^,\n]+)"
    RNA_types = re.findall(pattern, string)
    print("RNA_types:", list(sorted(set(RNA_types))))
    return list(sorted(set(RNA_types)))
def get_tag(tag_path, label_path, RNA, dataset):
    tag = np.load(tag_path)
    y = np.load(label_path)
    RNA_lst = GetRNAtype(dataset)
    idx = RNA_lst.index(RNA)
    arry_site = np.where(tag[:,0] == idx+6)
    filtered_tag = tag[arry_site,:]
    filtered_y = y[arry_site]
    return  filtered_tag,filtered_y
def cal_ratio(ary):
    sm = ary.sum(axis=0)
    all_sm = sm.sum()
    rat = sm/all_sm
    return sm,rat 

#human dataset statistic
# tag_path = "/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2/Test_fold0_X_tag.npy"
# label_path = "/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2/Test_fold0_y.npy"
# dataset = "/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2.fasta"
#mouse dataset statistic
tag_path = "/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_mouse_data_seq_deduplicated/Test_fold0_X_tag.npy"
label_path = "/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_mouse_data_seq_deduplicated/Test_fold0_y.npy"
dataset = "/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_mouse_data_seq_deduplicated.fasta"
X,y = get_tag(tag_path, label_path, "lncRNA", dataset)
sm,rat = cal_ratio(y)
print(y)
print(sm,rat)


RNA_types: ['lncRNA', 'mRNA', 'miRNA', 'ncRNA', 'pseudo', 'snRNA', 'snoRNA']
[[1 0 0 1 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]]
[8 7 0 5 0 0 0 0 0] [0.4  0.35 0.   0.25 0.   0.   0.   0.   0.  ]


In [10]:


RNA_type = GetRNAtype("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_mouse_data_seq_deduplicated.fasta")

RNA_types: ['lncRNA', 'mRNA', 'miRNA', 'ncRNA', 'pseudo', 'snRNA', 'snoRNA']


In [ ]:
X,y = get_tag("lincRNA")
sm,rat = cal_ratio(y)
print(sm,rat)

# Split the miRNA and lncRNA from the pooled dataset

In [8]:
#Because it has already been deduplicated from the pooled comprehensive dataset, we only need to filtered by sequence similarity

#spliting the lncRNA 
lnc_str = ""
tag = 0
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2.fasta", "r") as f1:
    for line in f1.readlines():
        # print(line)
        if line[0] == ">":
            RNA_type = line.split("RNA_category:")[1].split(",")[0]
            if RNA_type == "lncRNA" or RNA_type == "lincRNA":
                lnc_str += line
                tag = 1
            else:
                tag = 0
        else:
            seq = line
            if tag == 1:
                lnc_str += seq

#save the lncRNA to an independent file
with open("/home/sxr280/DeepRBPLoc/new_data/lncRNA/lncRNA_all_data_seq_splitfrom_pooled.fasta", "w") as f2:
    f2.write(lnc_str)
    


# Making all the subset of the allRNA filtered

In [25]:
#substitute the miRNA data
filtered_miRNA = []
with open("/home/sxr280/DeepRBPLoc/new_data/miRNA/miRNA_all_data_seq_filtered2.fasta", "r") as f1:
    for line in f1.readlines():
        if line[0] == ">":
            # print(line)
            ncbi_id = line.split("Gene_ID:")[1].split(":")[1].split(",")[0]
        else:
            seq = line.strip()
            filtered_miRNA.append(ncbi_id)

#substitute the lncRNA
filtered_lncRNA = []
with open("/home/sxr280/DeepRBPLoc/new_data/lncRNA/lncRNA_all_data_seq_deduplicated_splitfrom_pooled_filtered.fasta", "r") as f4:
    for line in f4.readlines():
        if line[0] == ">":
            # print(line)
            ncbi_id = line.split("Gene_ID:")[1].split(":")[1].split(",")[0]
        else:
            seq = line.strip()
            filtered_lncRNA.append(ncbi_id)



filtered_str = ""
filtered_miRNA_num = 0
filtered_lncRNA_num = 0
allRNA_dict = {}
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2.fasta", "r") as f2:
    for line in f2.readlines():
        if line[0] == ">":
            # print(line)
            id = line
            ncbi_id = line.split("Gene_ID:")[1].split(":")[1].split(",")[0]
            RNA_type = line.split("RNA_category:")[1].split(",")[0]

        else:
            seq = line
            allRNA_dict[id] = seq
            if RNA_type == "miRNA":
                if ncbi_id in filtered_miRNA:
                    filtered_str += id+seq
                else:
                    filtered_miRNA_num += 1
            elif RNA_type == "lncRNA" or RNA_type == "lincRNA":
                if ncbi_id in filtered_lncRNA:
                    filtered_str += id+seq
                else:
                    filtered_lncRNA_num += 1
            else:
                filtered_str += id+seq
print("number of miRNA that has been filtered:", filtered_miRNA_num)
print("number of lncRNA that has been filtered:", filtered_lncRNA_num)
#writing to a new file
with open("/home/sxr280/DeepRBPLoc/new_data/allRNA/allRNA_all_human_data_seq_mergedm3locall2_deduplicated2_filtermilnc.fasta", "w") as f3:        
    f3.write(filtered_str)
# print(filtered_dict.keys())

number of miRNA that has been filtered: 1036
number of lncRNA that has been filtered: 64


In [21]:
filtered_lncRNA

0

[0 0 0 0 0 0 0 0 0] [nan nan nan nan nan nan nan nan nan]


/tmp/ipykernel_1792635/1828579273.py:13: RuntimeWarning: invalid value encountered in true_divide
  rat = sm/all_sm
